# TCGA 데이터 원본 데이터로 바꾸기
1. 초반에 노션에 서은이가 올렸던 R의 TCGAbiolinks에서 gene 데이터 받는 방법으로 전에 다운 받았던 데이터 이용
   * "TCGA_BRCA_gene_expression.csv" 데이터로 전에 "target_gene 찾기 + 생존분석 연습"에서 사용
   * Q: 위 데이터는 read_count 데이터라고 볼 수 있는가?
   * A: STAR - Counts로 불러왔고 이는 RNA-seq데이터로 그렇다고 볼 수 있다.(출처: https://docs.gdc.cancer.gov/Encyclopedia/pages/STAR_Counts/?utm_source)
2. 이 데이터와 우리가 가진 TCGA_gene 데이터의 환자식별 코드가 겹치는게 있는지 확인
3. TCGA_BRCA_gene_expression.csv는 유전자형이 **Ensembl ID**로 되어 있고, 우리가 가진 TCGA_gene은 **유전자 심볼** 사용
4. https://biit.cs.ut.ee/gprofiler/convert 에서 Ensembl ID -> 유전자 심볼로 변환
5. 유전자 심볼 겹치는거 확인

In [1]:
import pandas as pd             
import numpy as np             
import matplotlib.pyplot as plt  
import seaborn as sns           

from sklearn.preprocessing import OneHotEncoder # 원핫인코딩
from sklearn.preprocessing import LabelEncoder # 라벨인코딩
from sklearn.model_selection import train_test_split  # 데이터 분할
from sklearn.preprocessing import StandardScaler      # 스케일링
from sklearn.impute import SimpleImputer              # 결측치 처리
from sklearn.ensemble import RandomForestClassifier   # 분류 모델
from sklearn.linear_model import LogisticRegression   # 분류 모델
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  # 모델 평가
from sklearn.linear_model import LinearRegression     # 회귀 모델
from sklearn.ensemble import RandomForestRegressor   # 회귀 모델
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score  # 모델 평가

# 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

## 한글 깨짐 방지
#pip install koreanize_matplotlib
import koreanize_matplotlib 

In [2]:
origin_df = pd.read_csv("C:/Users/User/OneDrive/바탕 화면/학회/25-1 의료/TCGA_BRCA_gene_expression.csv")

In [3]:
df=origin_df.copy()
df

,Unnamed: 0,TCGA-A2-A25D-01A-12R-A16F-07,TCGA-BH-A201-01A-11R-A14M-07,TCGA-AC-A23C-01A-12R-A169-07,TCGA-AR-A5QP-01A-11R-A28M-07,TCGA-C8-A12P-01A-11R-A115-07,TCGA-BH-A0W3-01A-11R-A109-07,TCGA-AC-A8OQ-01A-11R-A41B-07,TCGA-BH-A0H6-01A-21R-A056-07,TCGA-D8-A27V-01A-12R-A17B-07,...,TCGA-AC-A3TM-01A-11R-A22K-07,TCGA-A7-A26J-01A-11R-A277-07,TCGA-LL-A441-01A-11R-A24H-07,TCGA-A7-A13G-01A-11R-A13Q-07,TCGA-BH-A0E6-01A-11R-A034-07,TCGA-LL-A5YP-01A-21R-A28M-07,TCGA-AO-A03L-01A-41R-A056-07,TCGA-BH-A42T-01A-11R-A24H-07,TCGA-A2-A04W-01A-31R-A115-07,TCGA-AR-A24O-01A-11R-A169-07
0,ENSG00000000003.15,4474,3555,731,3960,3834,826,690,5534,13509,...,1153,599,5172,3299,7389,2300,955,2641,2718,1856
1,ENSG00000000005.6,9,30,113,50,0,8,55,29,15,...,5,0,68,24,20,7,3,40,1,58
2,ENSG00000000419.13,1785,2537,5947,2155,3161,1483,2456,1511,3060,...,1118,2843,4333,1465,1366,2524,3231,2311,1914,2184
3,ENSG00000000457.14,1362,1865,3118,1355,1411,1594,646,929,7099,...,573,2677,2584,5199,650,720,1763,1263,677,1206
4,ENSG00000000460.17,433,890,1063,474,637,645,764,300,1449,...,281,930,971,1257,1414,632,899,904,291,574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60655,ENSG00000288669.1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
60656,ENSG00000288670.1,150,590,754,332,354,238,347,333,1240,...,184,328,395,459,407,140,416,561,121,287
60657,ENSG00000288671.1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60658,ENSG00000288674.1,6,11,14,9,0,11,13,7,8,...,7,9,35,12,14,6,9,57,1,9


In [44]:
tcga = pd.read_csv("data/TCGA_gene_correspond.csv")
tcga

,Unnamed: 0,sample,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,REM1,RTN4RL2,...,pan.samplesID,cancer.type,Subtype_mRNA,Subtype_DNAmeth,Subtype_protein,Subtype_miRNA,Subtype_CNA,Subtype_Integrative,Subtype_other,Subtype_Selected
0,0,TCGA-AR-A5QQ-01,67.6890,7.3010,0.0000,91.0699,89.1384,75.0670,15.0357,16.5592,...,TCGA-AR-A5QQ-01,BRCA,Basal,NA_character_,-2147483648,NA_character_,NA_character_,NA_character_,NA_character_,BRCA.Basal
1,1,TCGA-D8-A1JA-01,43.2552,18.8283,3.7593,93.9684,98.9692,85.6586,13.9978,31.8358,...,TCGA-D8-A1JA-01,BRCA,Her2,NA_character_,-2147483648,NA_character_,NA_character_,NA_character_,NA_character_,BRCA.Her2
2,2,TCGA-BH-A0BQ-01,64.3948,11.5102,1.3625,95.1058,92.1185,81.8710,20.2297,26.4238,...,TCGA-BH-A0BQ-01,BRCA,LumA,NA_character_,-2147483648,NA_character_,NA_character_,NA_character_,NA_character_,BRCA.LumA
3,3,TCGA-BH-A0BT-01,60.4877,6.7588,0.0000,98.4396,89.0831,81.6024,13.5772,22.2999,...,TCGA-BH-A0BT-01,BRCA,LumA,NA_character_,-2147483648,NA_character_,NA_character_,NA_character_,NA_character_,BRCA.LumA
4,4,TCGA-A8-A06X-01,69.7906,1.5644,0.0000,95.2534,90.5039,77.8376,20.2904,21.0209,...,TCGA-A8-A06X-01,BRCA,LumB,NA_character_,-2147483648,NA_character_,NA_character_,NA_character_,NA_character_,BRCA.LumB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1213,1213,TCGA-A2-A3XT-01,92.7702,8.3572,7.3560,93.2273,79.4324,74.6340,20.0768,35.5264,...,TCGA-A2-A3XT-01,BRCA,Basal,NA_character_,-2147483648,NA_character_,NA_character_,NA_character_,NA_character_,BRCA.Basal
1214,1214,TCGA-B6-A0X7-01,83.8835,17.8496,0.9961,95.1562,86.3168,82.5972,22.8954,31.6751,...,TCGA-B6-A0X7-01,BRCA,LumA,NA_character_,-2147483648,NA_character_,NA_character_,NA_character_,NA_character_,BRCA.LumA
1215,1215,TCGA-BH-A1EV-11,66.2846,37.6901,0.0000,92.9888,94.8982,89.5175,23.0468,18.4840,...,TCGA-BH-A1EV-11,BRCA,Normal,NA_character_,-2147483648,NA_character_,NA_character_,NA_character_,NA_character_,BRCA.Normal
1216,1216,TCGA-3C-AALJ-01,87.8746,1.9467,0.0000,96.2317,86.2539,68.2448,21.0993,22.6852,...,TCGA-3C-AALJ-01,BRCA,LumB,NA_character_,-2147483648,NA_character_,NA_character_,NA_character_,NA_character_,BRCA.LumB


In [5]:
# 1. 열 이름에서 'TCGA-XX-XXXX-XX'까지만 추출
import re
trimmed_columns = [re.match(r'TCGA-\w\w-\w\w\w\w-\w\w', col).group() for col in origin_df.columns if re.match(r'TCGA-\w\w-\w\w\w\w-\w\w', col)]

# 2. tcga 데이터프레임의 sample 열과 비교
tcga_samples = tcga['sample'].tolist()

# 3. 교집합 구하기
common_samples = set(trimmed_columns) & set(tcga_samples)

# 4. 일치하는 샘플 수 출력
print(f"일치하는 샘플 수: {len(common_samples)}")


일치하는 샘플 수: 1215


In [11]:
import re

# 열 이름 중 'TCGA-XX-XXXX-XX' 형식 추출해서 교체
df.columns = [
    re.match(r'TCGA-\w\w-\w\w\w\w-\w\w', col).group()
    if re.match(r'TCGA-\w\w-\w\w\w\w-\w\w', col) else col
    for col in df.columns
]
`

,Unnamed: 0,TCGA-A2-A25D-01,TCGA-BH-A201-01,TCGA-AC-A23C-01,TCGA-AR-A5QP-01,TCGA-C8-A12P-01,TCGA-BH-A0W3-01,TCGA-AC-A8OQ-01,TCGA-BH-A0H6-01,TCGA-D8-A27V-01,...,TCGA-AC-A3TM-01,TCGA-A7-A26J-01,TCGA-LL-A441-01,TCGA-A7-A13G-01,TCGA-BH-A0E6-01,TCGA-LL-A5YP-01,TCGA-AO-A03L-01,TCGA-BH-A42T-01,TCGA-A2-A04W-01,TCGA-AR-A24O-01
0,ENSG00000000003.15,4474,3555,731,3960,3834,826,690,5534,13509,...,1153,599,5172,3299,7389,2300,955,2641,2718,1856
1,ENSG00000000005.6,9,30,113,50,0,8,55,29,15,...,5,0,68,24,20,7,3,40,1,58
2,ENSG00000000419.13,1785,2537,5947,2155,3161,1483,2456,1511,3060,...,1118,2843,4333,1465,1366,2524,3231,2311,1914,2184
3,ENSG00000000457.14,1362,1865,3118,1355,1411,1594,646,929,7099,...,573,2677,2584,5199,650,720,1763,1263,677,1206
4,ENSG00000000460.17,433,890,1063,474,637,645,764,300,1449,...,281,930,971,1257,1414,632,899,904,291,574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60655,ENSG00000288669.1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
60656,ENSG00000288670.1,150,590,754,332,354,238,347,333,1240,...,184,328,395,459,407,140,416,561,121,287
60657,ENSG00000288671.1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60658,ENSG00000288674.1,6,11,14,9,0,11,13,7,8,...,7,9,35,12,14,6,9,57,1,9


In [12]:
# 열 이름 중복 확인
duplicated_cols = df.columns[df.columns.duplicated()]

# 결과 출력
if len(duplicated_cols) > 0:
    print(f"중복된 열 이름 수: {len(duplicated_cols)}")
    print("중복 열 이름 예시:", duplicated_cols.unique().tolist()[:5])
else:
    print("df의 열 이름 내에서 중복은 없습니다.")


중복된 열 이름 수: 16
중복 열 이름 예시: ['TCGA-A7-A26I-01', 'TCGA-A7-A13D-01', 'TCGA-A7-A0DB-01', 'TCGA-A7-A26E-01', 'TCGA-AC-A3OD-01']


* 두 데이터셋의 샘플은 일치함 중복된 건 같은 사람 같은사람임.

# 유전자명 바꾸기

In [7]:
# Ensembl ID에서 버전 정보(.15 등) 제거
origin_genes = origin_df['Unnamed: 0'].str.split('.').str[0]

# 파일로 저장
origin_genes.to_csv('ensembl_ids.txt', index=False, header=False)


In [8]:
origin_genes


0        ENSG00000000003
1        ENSG00000000005
2        ENSG00000000419
3        ENSG00000000457
4        ENSG00000000460
              ...       
60655    ENSG00000288669
60656    ENSG00000288670
60657    ENSG00000288671
60658    ENSG00000288674
60659    ENSG00000288675
Name: Unnamed: 0, Length: 60660, dtype: object

# 위의 origin_genes를 토대로 변경 후 csv파일 이용

In [14]:
new_gene_name = pd.read_csv("C:/Users/User/Downloads/gProfiler_hsapiens_2025. 3. 27. 오후 1-00-21.csv")
new_gene_name

,initial_alias,converted_alias,name,description,namespace
0,ENSG00000000003,TSPAN6,TSPAN6,tetraspanin 6 [Source:HGNC Symbol;Acc:HGNC:11858],"ARRAYEXPRESS,ENSG"
1,ENSG00000000005,TNMD,TNMD,tenomodulin [Source:HGNC Symbol;Acc:HGNC:17757],"ARRAYEXPRESS,ENSG"
2,ENSG00000000419,DPM1,DPM1,dolichyl-phosphate mannosyltransferase subunit...,"ARRAYEXPRESS,ENSG"
3,ENSG00000000457,SCYL3,SCYL3,SCY1 like pseudokinase 3 [Source:HGNC Symbol;A...,"ARRAYEXPRESS,ENSG"
4,ENSG00000000460,FIRRM,FIRRM,FIGNL1 interacting regulator of recombination ...,"ARRAYEXPRESS,ENSG"
...,...,...,...,...,...
60658,ENSG00000288669,NaN,NaN,NaN,NaN
60659,ENSG00000288670,NaN,NaN,NaN,NaN
60660,ENSG00000288671,NaN,NaN,NaN,NaN
60661,ENSG00000288674,NaN,NaN,NaN,NaN


In [17]:
df['gene_symbol'] = new_gene_name["name"]
df = df.set_index('gene_symbol')


In [20]:
new_gene_name["name"].isnull().sum()

19714

In [18]:
df

,Unnamed: 0,TCGA-A2-A25D-01,TCGA-BH-A201-01,TCGA-AC-A23C-01,TCGA-AR-A5QP-01,TCGA-C8-A12P-01,TCGA-BH-A0W3-01,TCGA-AC-A8OQ-01,TCGA-BH-A0H6-01,TCGA-D8-A27V-01,...,TCGA-AC-A3TM-01,TCGA-A7-A26J-01,TCGA-LL-A441-01,TCGA-A7-A13G-01,TCGA-BH-A0E6-01,TCGA-LL-A5YP-01,TCGA-AO-A03L-01,TCGA-BH-A42T-01,TCGA-A2-A04W-01,TCGA-AR-A24O-01
gene_symbol,,,,,,,,,,,,,,,,,,,,,
TSPAN6,ENSG00000000003.15,4474,3555,731,3960,3834,826,690,5534,13509,...,1153,599,5172,3299,7389,2300,955,2641,2718,1856
TNMD,ENSG00000000005.6,9,30,113,50,0,8,55,29,15,...,5,0,68,24,20,7,3,40,1,58
DPM1,ENSG00000000419.13,1785,2537,5947,2155,3161,1483,2456,1511,3060,...,1118,2843,4333,1465,1366,2524,3231,2311,1914,2184
SCYL3,ENSG00000000457.14,1362,1865,3118,1355,1411,1594,646,929,7099,...,573,2677,2584,5199,650,720,1763,1263,677,1206
FIRRM,ENSG00000000460.17,433,890,1063,474,637,645,764,300,1449,...,281,930,971,1257,1414,632,899,904,291,574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,ENSG00000288669.1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
NaN,ENSG00000288670.1,150,590,754,332,354,238,347,333,1240,...,184,328,395,459,407,140,416,561,121,287
NaN,ENSG00000288671.1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# gene 이름 변경 후 가지고 있는 TCGA gene이랑 R에서 받은거랑 gene이름이 겹치는게 있는지 확인

In [24]:
# tcga 데이터의 gene symbol 목록 (열 이름 중 샘플 정보 등 제외하고)
# 일반적으로 'sample' 같은 메타 정보 열은 제외하길 원함
tcga_gene_symbols = [col for col in tcga.columns if col not in ['Unnamed: 0', 'sample', 'pan.samplesID', 
                                                                 'cancer.type', 'Subtype_mRNA', 'Subtype_DNAmeth', 
                                                                 'Subtype_protein', 'Subtype_miRNA', 'Subtype_CNA', 
                                                                 'Subtype_Integrative', 'Subtype_other', 'Subtype_Selected']]

# origin_df의 행 인덱스 (gene symbol) 중에서 tcga의 열 이름과 일치하는 것들
common_genes = df.index.intersection(tcga_gene_symbols)

# 결과 출력
print(f"df와 tcga에서 공통으로 존재하는 gene symbol 개수: {len(common_genes)}")

# 공통 gene만 추출
filtered_df = df.loc[common_genes]
print("우리가 가지고 있던 gene이 거의 대부분 존재했다.")

df와 tcga에서 공통으로 존재하는 gene symbol 개수: 16399
우리가 가지고 있떤 gene이 거의 대부분 존재했다.


In [25]:
filtered_df

,Unnamed: 0,TCGA-A2-A25D-01,TCGA-BH-A201-01,TCGA-AC-A23C-01,TCGA-AR-A5QP-01,TCGA-C8-A12P-01,TCGA-BH-A0W3-01,TCGA-AC-A8OQ-01,TCGA-BH-A0H6-01,TCGA-D8-A27V-01,...,TCGA-AC-A3TM-01,TCGA-A7-A26J-01,TCGA-LL-A441-01,TCGA-A7-A13G-01,TCGA-BH-A0E6-01,TCGA-LL-A5YP-01,TCGA-AO-A03L-01,TCGA-BH-A42T-01,TCGA-A2-A04W-01,TCGA-AR-A24O-01
gene_symbol,,,,,,,,,,,,,,,,,,,,,
TSPAN6,ENSG00000000003.15,4474,3555,731,3960,3834,826,690,5534,13509,...,1153,599,5172,3299,7389,2300,955,2641,2718,1856
TNMD,ENSG00000000005.6,9,30,113,50,0,8,55,29,15,...,5,0,68,24,20,7,3,40,1,58
DPM1,ENSG00000000419.13,1785,2537,5947,2155,3161,1483,2456,1511,3060,...,1118,2843,4333,1465,1366,2524,3231,2311,1914,2184
SCYL3,ENSG00000000457.14,1362,1865,3118,1355,1411,1594,646,929,7099,...,573,2677,2584,5199,650,720,1763,1263,677,1206
FGR,ENSG00000000938.13,928,711,460,237,468,251,562,252,495,...,190,101,1371,190,415,491,324,744,224,402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CCDC39,ENSG00000284873.1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
LRTOMT,ENSG00000284930.1,0,2,0,1,1,1,2,0,1,...,0,0,2,2,1,3,1,0,0,2
ARHGAP11B,ENSG00000285080.1,0,1,0,0,0,0,1,3,1,...,0,0,2,5,0,0,0,0,0,2


# pathway로 추린 gene이랑 매핑

In [28]:
final_gene = pd.read_csv("C:/Users/User/Downloads/#_final_filtered_gene_expression.csv")
final_gene

,Unnamed: 0,A4GALT,AACS,AADAT,AASDHPPT,AASS,ABAT,ABCA1,ABCC1,ABCC2,...,ZFYVE9,ZMAT3,ZMPSTE24,ZNF274,ZNF385A,ZNF516,ZNF76,ZNFX1,ZNRF3,ZYX
0,ACH-000242,1815,6327,1055,2947,1115,3106,1021,6828,28,...,6384,1418,7417,1213,5812,1225,2784,4722,3584,4962
1,ACH-000327,7,11985,100,2562,177,25,165,4013,17436,...,1463,2750,7519,2915,706,1529,2074,2374,1320,2120
2,ACH-000233,4,2167,968,2342,19,376,4387,4050,473,...,176,2593,2512,1285,401,967,3148,4983,501,18346
3,ACH-000528,775,1911,403,5141,6,2182,6,2902,207,...,4992,5835,6369,1148,698,713,2125,3802,4736,2721
4,ACH-001655,600,2801,594,6196,3127,8,4155,5686,95,...,1495,1718,8411,4374,1581,1737,2487,6274,682,43403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688,ACH-000114,736,3213,1249,2760,2658,1703,998,12183,117,...,1179,1765,2767,1668,424,1102,2144,9229,672,3662
689,ACH-001578,704,2634,775,6986,10798,1195,9868,8418,110,...,12783,7582,11966,2676,125,1950,2687,2023,1188,4002
690,ACH-000973,80,3634,850,5000,4193,712,197,5280,51,...,3659,1756,6007,2048,586,831,2120,3704,2563,12367
691,ACH-000750,2,1982,451,5667,3663,5,4841,4344,131,...,1888,3418,3902,2158,312,2549,1592,5708,785,12522


In [29]:
# 교집합: filtered_data의 인덱스 ∩ final_gene의 열이름
common_genes = filtered_df.index.intersection(final_gene.columns)

# 해당하는 행만 남기기
filtered_data_matched = filtered_df.loc[common_genes]

# 결과 확인
print(f"남은 행 개수: {filtered_data_matched.shape[0]}")


남은 행 개수: 2768


In [31]:
filtered_data_matched.drop("Unnamed: 0",axis=1)

,TCGA-A2-A25D-01,TCGA-BH-A201-01,TCGA-AC-A23C-01,TCGA-AR-A5QP-01,TCGA-C8-A12P-01,TCGA-BH-A0W3-01,TCGA-AC-A8OQ-01,TCGA-BH-A0H6-01,TCGA-D8-A27V-01,TCGA-D8-A27M-01,...,TCGA-AC-A3TM-01,TCGA-A7-A26J-01,TCGA-LL-A441-01,TCGA-A7-A13G-01,TCGA-BH-A0E6-01,TCGA-LL-A5YP-01,TCGA-AO-A03L-01,TCGA-BH-A42T-01,TCGA-A2-A04W-01,TCGA-AR-A24O-01
DPM1,1785,2537,5947,2155,3161,1483,2456,1511,3060,1446,...,1118,2843,4333,1465,1366,2524,3231,2311,1914,2184
CFH,2445,4352,4609,2440,4472,419,2113,1567,18882,9061,...,798,3358,3009,2195,997,3520,2690,1078,1053,7647
GCLC,2278,2472,2265,2637,1601,3506,1981,2964,3043,3206,...,702,2180,3026,2195,843,771,1235,1359,703,2423
NFYA,2511,3796,4388,1914,1585,2118,4660,2478,8263,3552,...,1305,2915,9294,2583,2632,2189,4038,2826,1271,3644
CYP51A1,75,360,114,55,95,141,126,98,222,369,...,1,253,172,348,26,50,98,107,111,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ORAI1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PIGW,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MARCKS,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACACA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 데이터 3개 비교

In [45]:
tcga.sort_values("sample")

,Unnamed: 0,sample,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,REM1,RTN4RL2,...,pan.samplesID,cancer.type,Subtype_mRNA,Subtype_DNAmeth,Subtype_protein,Subtype_miRNA,Subtype_CNA,Subtype_Integrative,Subtype_other,Subtype_Selected
76,76,TCGA-3C-AAAU-01,82.1139,12.5746,0.0000,93.4803,90.6021,78.2274,19.8997,41.5484,...,TCGA-3C-AAAU-01,BRCA,LumA,NA_character_,-2147483648,NA_character_,NA_character_,NA_character_,NA_character_,BRCA.LumA
247,247,TCGA-3C-AALI-01,63.5096,1.7447,1.7447,94.8051,81.6512,77.9069,18.5729,33.9168,...,TCGA-3C-AALI-01,BRCA,Her2,NA_character_,-2147483648,NA_character_,NA_character_,NA_character_,NA_character_,BRCA.Her2
1216,1216,TCGA-3C-AALJ-01,87.8746,1.9467,0.0000,96.2317,86.2539,68.2448,21.0993,22.6852,...,TCGA-3C-AALJ-01,BRCA,LumB,NA_character_,-2147483648,NA_character_,NA_character_,NA_character_,NA_character_,BRCA.LumB
879,879,TCGA-3C-AALK-01,75.2814,12.3397,0.0000,95.0692,88.5563,76.0598,22.8006,25.7770,...,TCGA-3C-AALK-01,BRCA,LumA,NA_character_,-2147483648,NA_character_,NA_character_,NA_character_,NA_character_,BRCA.LumA
427,427,TCGA-4H-AAAK-01,80.3194,10.5990,0.0000,94.7687,87.6137,74.4914,14.4550,28.7310,...,TCGA-4H-AAAK-01,BRCA,LumA,NA_character_,-2147483648,NA_character_,NA_character_,NA_character_,NA_character_,BRCA.LumA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,573,TCGA-WT-AB44-01,61.0137,12.2633,0.0000,96.4433,81.1914,73.8567,12.9187,24.9199,...,TCGA-WT-AB44-01,BRCA,LumA,NA_character_,-2147483648,NA_character_,NA_character_,NA_character_,NA_character_,BRCA.LumA
133,133,TCGA-XX-A899-01,74.7180,23.5763,8.2977,94.8275,87.7299,80.0239,20.7012,24.1069,...,TCGA-XX-A899-01,BRCA,LumA,NA_character_,-2147483648,NA_character_,NA_character_,NA_character_,NA_character_,BRCA.LumA
173,173,TCGA-XX-A89A-01,83.7315,24.9874,0.0000,96.1680,87.0862,75.0488,20.4410,24.8591,...,TCGA-XX-A89A-01,BRCA,LumA,NA_character_,-2147483648,NA_character_,NA_character_,NA_character_,NA_character_,BRCA.LumA
1136,1136,TCGA-Z7-A8R5-01,77.9493,10.0023,0.0000,95.4483,86.7331,75.0414,34.3559,21.3544,...,TCGA-Z7-A8R5-01,BRCA,LumA,NA_character_,-2147483648,NA_character_,NA_character_,NA_character_,NA_character_,BRCA.LumA


In [43]:
filtered_sort = filtered_data_matched.sort_index()
filtered_sort.sort_index(axis=1)

,TCGA-3C-AAAU-01,TCGA-3C-AALI-01,TCGA-3C-AALJ-01,TCGA-3C-AALK-01,TCGA-4H-AAAK-01,TCGA-5L-AAT0-01,TCGA-5L-AAT1-01,TCGA-5T-A9QA-01,TCGA-A1-A0SB-01,TCGA-A1-A0SD-01,...,TCGA-UU-A93S-01,TCGA-V7-A7HQ-01,TCGA-W8-A86G-01,TCGA-WT-AB41-01,TCGA-WT-AB44-01,TCGA-XX-A899-01,TCGA-XX-A89A-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01,Unnamed: 0
A4GALT,236,320,686,1328,858,699,355,826,874,1255,...,93,2603,636,2773,473,1386,1438,841,2431,ENSG00000128274.17
AACS,3446,1683,1837,2244,2158,1526,1208,2400,5551,2612,...,3211,2981,1351,1190,837,3343,2082,1851,2288,ENSG00000081760.17
AADAT,26,61,86,170,174,80,53,261,550,161,...,21,61,87,69,59,134,118,90,93,ENSG00000109576.14
AASDHPPT,2120,1186,810,1477,1017,1004,778,1164,1961,1761,...,746,648,1375,395,912,1323,916,768,1289,ENSG00000149313.11
AASS,163,163,105,486,983,881,445,308,1028,1111,...,81,129,1301,105,654,1451,392,455,147,ENSG00000008311.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZNF516,4481,175,185,1949,1721,481,681,1712,2091,2468,...,40,519,1151,1251,379,2029,1475,824,2297,ENSG00000101493.11
ZNF76,2865,2428,840,2236,1975,1496,1564,2562,1501,1886,...,1563,1930,3093,1669,1488,2236,2295,1907,2453,ENSG00000065029.15
ZNFX1,5772,4781,1402,3574,4250,2743,2981,3352,2691,13115,...,2419,1631,2816,2260,1968,4716,3605,1661,3641,ENSG00000124201.15
ZNRF3,2328,772,307,863,907,477,123,248,1061,1497,...,254,187,443,358,192,824,467,244,1413,ENSG00000183579.16


In [36]:
final_gene.T

,0,1,2,3,4,5,6,7,8,9,...,683,684,685,686,687,688,689,690,691,692
Unnamed: 0,ACH-000242,ACH-000327,ACH-000233,ACH-000528,ACH-001655,ACH-000792,ACH-000570,ACH-000351,ACH-000769,ACH-000828,...,ACH-000422,ACH-000200,ACH-000046,ACH-000968,ACH-000830,ACH-000114,ACH-001578,ACH-000973,ACH-000750,ACH-000052
A4GALT,1815,7,4,775,600,910,63,2147,132,162,...,285,43,674,5,16,736,704,80,2,2048
AACS,6327,11985,2167,1911,2801,3952,2128,3838,2692,3488,...,2320,2242,4046,1830,7550,3213,2634,3634,1982,3958
AADAT,1055,100,968,403,594,356,456,249,774,24,...,261,432,539,1282,1322,1249,775,850,451,703
AASDHPPT,2947,2562,2342,5141,6196,1002,3196,3746,3989,1273,...,4779,3789,3244,1168,2613,2760,6986,5000,5667,6790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZNF516,1225,1529,967,713,1737,4056,1992,1060,2033,604,...,2032,1208,2022,2665,5032,1102,1950,831,2549,3782
ZNF76,2784,2074,3148,2125,2487,2076,2557,1444,1775,2200,...,2348,1156,1262,4929,2879,2144,2687,2120,1592,2874
ZNFX1,4722,2374,4983,3802,6274,2354,5748,15668,719,4508,...,7223,6588,2958,4222,1180,9229,2023,3704,5708,3216
ZNRF3,3584,1320,501,4736,682,1343,1407,1468,8553,1809,...,1381,1638,1674,2540,1802,672,1188,2563,785,2076


In [46]:
filtered_sort.to_csv("C:/Users/User/BAF-의료\data/final_tcga.csv")